## Vamos a usar as seguintes bibliotecas

In [1]:
#Load required libraries
import numpy as np
import pandas as pd
#For displaying complete rows info
pd.options.display.max_colwidth=500
import tensorflow as tf
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
print(tf.__version__)


2.6.2


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#import os
#workdir_path = '/content/drive/My Drive' #MODIFICAR!!! #Caminho para o seu workspace
#os.chdir(workdir_path)

### Carregamos os artigos

In [2]:
#Load data into pandas dataframe
df=pd.read_csv("folha.csv",encoding="utf8")

In [3]:
df.head(2)

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral","Com a possibilidade de uma condenação impedir sua candidatura em 2018, o ex-presidente Luiz Inácio Lula da Silva fez, nesta segunda (9), um discurso inflamado contra a Lava Jato, no qual disse saber que está ""lascado"", exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que, mesmo fora da disputa pelo Planalto, será um cabo eleitoral expressivo para a sucessão de Michel Temer. Segundo o petista, réu em sete ações penais, o objetivo de Moro é impedir sua candidatura no ano que vem, d...",2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/1925743-lula-diz-que-esta-lascado-mas-que-ainda-tem-forca-como-cabo-eleitoral.shtml
1,"'Decidi ser escrava das mulheres que sofrem', diz cantora Oumou Sangaré","Para Oumou Sangaré, cantora e ativista malinesa, o sofrimento da mulher é universal. ""Canto com todas as minhas forças e vejo mulheres chorando ao meu redor"", disse durante um encontro na Casa da Cultura de Paraty, no Rio de Janeiro, neste domingo (8). Herdeira da tradição musical de sua família –sua avó foi uma ""grande estrela"" do país africano–, ela conta que começou a cantar na rua para ajudar nas finanças de casa após o divórcio dos pais, que deixou seis filhos dependentes de sua mãe, t...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10/1925745-decidi-ser-escrava-das-mulheres-que-sofrem-diz-cantora-oumou-sangare.shtml


In [4]:
print(df["title"][0],"\n",df["text"][0])

Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral 
 Com a possibilidade de uma condenação impedir sua candidatura em 2018, o ex-presidente Luiz Inácio Lula da Silva fez, nesta segunda (9), um discurso inflamado contra a Lava Jato, no qual disse saber que está "lascado", exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que, mesmo fora da disputa pelo Planalto, será um cabo eleitoral expressivo para a sucessão de Michel Temer.  Segundo o petista, réu em sete ações penais, o objetivo de Moro é impedir sua candidatura no ano que vem, desidratando-o, inclusive, no apoio a um nome alternativo, como o do ex-prefeito de São Paulo Fernando Haddad (PT), caso ele não possa concorrer à Presidência.  "Eu sei que tô lascado, todo dia tem um processo. Eu não quero nem que Moro me absolva, eu só quero que ele peça desculpas", disse Lula durante um seminário sobre educação em Brasília. "Eles [investigadores] chegam a dizer: 'Ah, se o Lula não for candidato, ele não vai

## removemos nans

In [5]:
df.drop_duplicates(subset=["text"],inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

### WORD2VEC MODEL Usando Gensim

In [6]:
import gensim
import string
import re

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/LeonardoLins/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Deixamos todo mundo minúscula e tiramos a pontuação 

In [8]:
import re
import string

alphanumeric = lambda x: re.sub(r"""\w*\d\w*""", ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['text'] = df.text.map(alphanumeric).map(punc_lower)
df.head()

,title,text,date,category,subcategory,link
0,\n\t\tDa escola ao 1º emprego: como buscar um caminho profissional sem ter (ao menos ainda) um diploma universitário\n\t\t,da bbc brasil o trimestre de maio a julho deste ano foi encerrado com um índice de desemprego na casa de o percentual assusta mas é menor do que os apurados no fim do período imediatamente anterior os dados mostram um quadro preocupante mas que sugere que o pior da crise pode já ter ficado para trás só que a situação dos trabalhadores entre e anos é mais complicada do que a da média segundo a mesma pesquisa do ibge nada menos do que dos jovens nessa faixa etária...,2017-09-10,sobretudo,carreiras,http://www1.folha.uol.com.br/sobretudo/carreiras/2017/10/1925565-da-escola-ao-1-emprego-como-buscar-um-caminho-profissional-sem-ter-ao-menos-ainda-um-diploma-universitario.shtml
1,"50 anos depois da morte, o Che é apenas uma foto na camiseta",ernesto rafael guevara de la serna o homem morreu assassinado na bolívia há exatos anos neste de outubro mas o che o revolucionário morrera antes na cuba em que estão enterrados seus restos o che perdeu internamente a batalha de ideias pelo modelo econômico que deveria ser implantado na ilha prevaleceu o modelo soviético imposto pela necessidade de preservar o financiamento que a união soviética concedia a cuba escreve por exemplo para el país o historiador cubano r...,2017-09-10,colunas,clovisrossi,http://www1.folha.uol.com.br/colunas/clovisrossi/2017/10/1925562-50-anos-depois-da-morte-o-che-e-apenas-uma-foto-na-camiseta.shtml
2,"Afinal, nosso voto tem preço?",a reforma política aprovada pelo congresso deixa claro o seu objetivo impedir a renovação os mecanismos implementados servirão basicamente para resolver o maior problema dos grandes partidos políticos a falta de dinheiro para financiar as suas campanhas essa demanda desesperada por recursos públicos revela a natureza e a qualidade da nossa representatividade os partidos não conseguem convencer o cidadão dos seus planos para o país que justifique um investimento nas suas propostas ao ...,2017-09-10,colunas,joao-amoedo,http://www1.folha.uol.com.br/colunas/joao-amoedo/2017/10/1925545-afinal-nosso-voto-tem-preco.shtml
3,"Cientista dá psicodélico para minicérebros, e eles gostam",a batalha de jovens neurocientistas para reabilitar drogas psicodélicas dá mais um passo nesta segunda feira desta vez com a ajuda de minicérebros um grupo do rio de janeiro tratou essas esferas de células neurais com dmt e constatou uma série de efeitos benéficos a pesquisa aparece no periódico scientific reports do grupo nature sob o título alterações de curto prazo no proteoma de organoides cerebrais humanos induzidas por meo dmt o composto meo dmt pertence à da famíli...,2017-09-10,colunas,marceloleite,http://www1.folha.uol.com.br/colunas/marceloleite/2017/10/1925096-cientista-da-psicodelico-para-minicerebros-e-eles-gostam.shtml
4,Reestruturação da Sabesp deverá ser levada à Justiça por setor de limpeza,associações de empresas de limpeza urbana e resíduos sólidos estudam entrar com uma ação contra a reestruturação societária da sabesp aprovada em setembro o temor é que a companhia paulista passe a oferecer novos serviços —como coleta ou operação de aterros— e aproveite sua capilaridade entre os municípios para ter uma concorrência desleal possivelmente seria uma adi ação direta de inconstitucionalidade ou uma ação junto ao cade órgão que regula a concorrência afirma wladimir rib...,2017-09-10,colunas,mercadoaberto,http://www1.folha.uol.com.br/colunas/mercadoaberto/2017/10/1925191-reestruturacao-da-sabesp-devera-ser-levada-a-justica-por-setor-de-limpeza.shtml


## Tokenizamos e guardamos todo em um unico repositorio

In [9]:
from nltk.tokenize import word_tokenize

## transformando em uma lista de tokens

#### Aplicamos o modelo word2vec. Por default o modelo vem com uma janela de 5. Troque e compare os resultados com os dados na aula agregue window= ??? e 50 dimensões.

In [10]:
corpus = df.text.apply(lambda x:word_tokenize(x))

In [11]:
model = gensim.models.Word2Vec(corpus, window=3, vector_size=100) # size = 100

In [12]:
model.wv['fluminense'] #vetor com 100 posições, pode ser com qualquer valor.

array([ 0.2636147 ,  0.65593046,  0.00799805,  0.2070306 ,  0.6517422 ,
       -0.9403182 , -0.03978563,  1.210654  , -0.9878935 ,  0.01232203,
        0.00344867,  0.23950419, -1.1713128 , -0.28245938, -0.16706863,
       -0.29117787, -0.02720916, -0.43869367,  0.70226115, -0.50060475,
       -0.6800562 ,  1.3212764 ,  0.9921837 ,  0.16720484, -0.7411293 ,
        0.14260462,  0.3613299 , -1.3521957 , -0.45617908,  1.0293871 ,
       -0.20602031,  0.57930523,  0.01221772, -1.3924181 ,  0.83379257,
       -1.1919478 ,  0.04067051,  0.8620389 , -0.3266618 ,  0.83124244,
        0.5161149 , -0.2970001 ,  0.16783258,  0.08364815, -1.3535153 ,
        1.216139  ,  0.4905827 , -1.5111136 ,  0.698743  ,  0.4880476 ,
        1.1044104 ,  0.46687508, -1.3265029 , -0.49161437, -0.7723586 ,
       -1.9111967 , -0.16028227, -0.44543266, -0.5156889 , -2.0125527 ,
        0.6659786 , -0.29586506, -0.5731451 , -0.7506465 , -0.5536642 ,
        0.05319941,  0.01309001,  0.8342715 , -1.5442085 ,  0.35

In [13]:
model.wv.most_similar("seleção") #distancia vetorial entre pontos mais similares

[('equipe', 0.7990208864212036),
 ('natação', 0.7541236281394958),
 ('delegação', 0.7075871229171753),
 ('libertadores', 0.7010970115661621),
 ('cbf', 0.6900712847709656),
 ('torcida', 0.6796157956123352),
 ('competição', 0.6743165254592896),
 ('fifa', 0.6611928939819336),
 ('semifinal', 0.6492189764976501),
 ('partida', 0.6424492597579956)]

In [14]:
model.wv.most_similar("propina")

[('propinas', 0.7173401117324829),
 ('empreiteira', 0.6591830849647522),
 ('suborno', 0.6528440713882446),
 ('odebrecht', 0.6420326828956604),
 ('indenização', 0.6302095055580139),
 ('petrobras', 0.6067284941673279),
 ('oas', 0.6043866276741028),
 ('pf', 0.5961465835571289),
 ('multa', 0.5902708172798157),
 ('pensão', 0.5792675018310547)]

In [15]:
model.wv.most_similar("gilmar")

[('zavascki', 0.7483001351356506),
 ('gabus', 0.728987455368042),
 ('lewandowski', 0.7208383679389954),
 ('fachin', 0.7190772294998169),
 ('teori', 0.683875560760498),
 ('barroso', 0.6820991635322571),
 ('geddel', 0.675216019153595),
 ('mendes', 0.6696389317512512),
 ('stf', 0.6501621007919312),
 ('supremo', 0.644054651260376)]

In [16]:
model.wv.most_similar("petrolão")

[('mensalão', 0.8578479886054993),
 ('escândalo', 0.7335081100463867),
 ('cartel', 0.655720055103302),
 ('stf', 0.6485605835914612),
 ('petismo', 0.638002872467041),
 ('carf', 0.6244016885757446),
 ('delcídio', 0.6164947152137756),
 ('fbi', 0.6140791773796082),
 ('tcu', 0.6134937405586243),
 ('suborno', 0.6067258715629578)]

In [17]:
model.wv.most_similar("azul")

[('verde', 0.8184996247291565),
 ('vermelha', 0.7660054564476013),
 ('amarela', 0.7543870210647583),
 ('marrom', 0.729994535446167),
 ('amarelo', 0.7208455801010132),
 ('batata', 0.7017553448677063),
 ('vermelho', 0.6948390007019043),
 ('cinza', 0.6891679167747498),
 ('roxo', 0.6871031522750854),
 ('jeans', 0.6674363017082214)]

In [19]:
model.wv.most_similar("pele")

[('barriga', 0.7912265062332153),
 ('garganta', 0.7672030329704285),
 ('nuca', 0.7306966185569763),
 ('cintura', 0.7266094088554382),
 ('testa', 0.7202082276344299),
 ('bagagem', 0.7200389504432678),
 ('geladeira', 0.7179006934165955),
 ('superfície', 0.715384304523468),
 ('cabeça', 0.7118871212005615),
 ('calcinha', 0.7105823159217834)]